# Hello everyone, 
The community contributions folder is super motivating. Thanks to Ed for democratising learning with this great idea of sharing. The below small piece is my novice attempt in summarizing content from wikipedia page. It is pretty straightforward, but a good learning exercise for me nevertheless. 

In [ ]:
!pip install wikipedia

In [ ]:
# Import statements
import os
import requests
from dotenv import load_dotenv
from IPython.display import Markdown, display
from openai import OpenAI
import wikipedia
import warnings

In [ ]:
#To supress a warning from wikipedia module when there are multiple options.
warnings.filterwarnings("ignore", category=UserWarning, module="wikipedia")

In [ ]:
# Load environment variables in a file called .env

load_dotenv()
api_key = os.getenv('OPENAI_API_KEY')

# Check the key

if not api_key:
    print("No API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!")
elif not api_key.startswith("sk-proj-"):
    print("An API key was found, but it doesn't start sk-proj-; please check you're using the right key - see troubleshooting notebook")
elif api_key.strip() != api_key:
    print("An API key was found, but it looks like it might have space or tab characters at the start or end - please remove them - see troubleshooting notebook")
else:
    print("API key found and looks good so far!")

In [ ]:
openai = OpenAI()

# If this doesn't work, try Kernel menu >> Restart Kernel and Clear Outputs Of All Cells, then run the cells from the top of this notebook down.
# If it STILL doesn't work (horrors!) then please see the troubleshooting notebook, or try the below line instead:
# openai = OpenAI(api_key="your-key-here-starting-sk-proj-")

In [ ]:
class Wikipedia:
    def __init__(self, searchText):
        """
        Create this object to extract the summary of wikipedia page for a text entered by user
        """
        self.searchText = searchText
        self.summary_text = None
        self.user_prompt = None
        
        self._fetch_summary()

    def _fetch_summary(self):
        """
        Fetches the summary from wikipedia page based on user entered search text and sets user prompt accordingly
        """
        try:
            # Try to get the summary of the text from Wikipedia based on user entered text. Using starightforward summary module in wikipedia.
            self.summary_text = wikipedia.summary(self.searchText)
            self.user_prompt = f"You are looking a summary extract from a wikipedia page. The content is as follows\n {self.summary_text}.\nProvide \
            a summary taking key points from each sections listed on the page"
        except wikipedia.DisambiguationError as e:
            #Modify user and system prompts if there are multiple options for a user search text
            self.user_prompt = f"You have received quite a few options {e.options} for the keyword {self.searchText}. Please request user to choose one of them"
        except wikipedia.PageError:
            #To handle when there is no page
            self.user_prompt = f"There is no wiki page for {self.searchText}. Apparently it is not your fault!"
        except Exception as e:
            # To handle any other exceptions
            self.user_prompt = f"Sorry, something seems to be wrong on my end. Please try again later"

In [ ]:
# Our by-now familiar sumamrize function
def summarize(searchText):
    wiki = Wikipedia(searchText)
    system_prompt = f"You are an assitant trying to summarize content from Wikipedia. You will have three scenarios to handle your responses \
        1. You will have the summary text content and you will just show that to user\
        2. You will have multiple options for the user entered keyword, and you will respond by asking user to choose from that and request again \
        3. You will not have the content due to a page not found error. Respond accordingly.\
        Respond all of these in Markdown format."
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": wiki.user_prompt}
    ]
    response = openai.chat.completions.create(
        model = "gpt-4o-mini",
        messages = messages
    )
    return response.choices[0].message.content



In [ ]:
summary = summarize("mukhari")
display(Markdown(summary))